# import

In [ ]:
!git clone https://github.com/H999/GeneticCNN-torch.git
%cd GeneticCNN-torch/
!git checkout H999/Population
%cd geneticCNN/

In [ ]:
import torch
import random
import numpy as np
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from tqdm.auto import tqdm
from torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
batch_size = 64

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = datasets.MNIST('data', train=True, download=True, transform=transform)
testset = datasets.MNIST('data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True)


def length(x): return len(x) * x.batch_size


print("Total train examples: ", length(train_loader))
print("Total test examples: ", length(test_loader))


In [ ]:
def set_seed(seed):
    # Set random seed for reproducibility
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = True


seed = 1234
set_seed(seed)


# class Stage

In [ ]:
from Stage import Stage

# class Stages

In [ ]:
from Stages import Stages

# class Individual

In [ ]:
from Individual import Individual

# class Population

In [ ]:
from Population import Population

# Train & test model for Individual

In [ ]:
def train(model, device, train_loader, optimizer, epoch):
    model.to(device)
    model.train()
    pbar = tqdm(train_loader, leave=False, desc="Train Epoch {}".format(epoch))
    for data, target in pbar:
        data, target = data.to(device), target.to(device)
        model.zero_grad()
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_postfix({'loss': '{:.6f}'.format(loss.item())})


def test(model, device, test_loader, epoch):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set {}: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        epoch, test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [ ]:
a = Individual(2)
optimizer = torch.optim.SGD(a.parameters(), lr=0.01, momentum=0.9)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)
EPOCHS = 5


In [ ]:
for epoch in tqdm(range(EPOCHS)):
    train(a, device, train_loader, optimizer, epoch)
    test(a, device, test_loader, epoch)
    scheduler.step()


# test

In [ ]:
# a=Stage(5,'1234567890')
# a=Stage(5,'1101100101')
# a=Stage(5,'1010000000')
# a=Stage(5,'1100000001')
# a=Stage(5,'0010001111')
# a=Stage(5,'0010001110')
# a=Stage(5)
# a=Stage(3)
# a=Stages(5)
# a=Stages(3)
# a=Individual(2)
a = Population(train_loader, test_loader,
               torch.optim.SGD, {'lr': 0.01, 'momentum': 0.9}, F.nll_loss,
               torch.optim.lr_scheduler.ExponentialLR, {'gamma': 0.95})
a.run()

In [ ]:
a = Population(train_loader, test_loader,
               torch.optim.Adam, {}, F.cross_entropy,
               torch.optim.lr_scheduler.ExponentialLR, {'gamma': 0.95})
a.run()

# test on tensorboard

In [ ]:
writer = SummaryWriter()
# writer.add_graph(a,images)

In [ ]:
# writer.close()

In [ ]:
# import time
# for i in range(1024):
#     print("{0:010b}".format(i))
#     try:
#         time.sleep(5)
#         writer = SummaryWriter()
#         writer.add_graph(Stage(5,"{0:010b}".format(i)),images)
#         writer.close()
#     except Exception as e:
#         print(e)

In [ ]:
!kill 

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir runs